# Week 08 Solution

Given classes `Guest` and `HotelAlphabetical` below, design and implement the following methods.

- `HotelAlphabetical.exists(self, guest_name:str) -> bool:` that returns true if a guest with the specified name is present in the hotel and false otherwise.

* `HotelAlphabetical.remove(self, guest_name:str) -> Guest | None:` that removes a guest, if present, from the hotel. If the removal is successful, the guest object is returned; otherwise the method returns `None`.


In [5]:
class Guest:
    """Class representing guests for our Hotel Alphabetical. These objects
    are essentially nodes in a linked list. Each hotel room is effectively a
    linked list of guests, placed in that room based on some hashing function.
    """

    def __init__(self, name: str):
        self.name = name
        self.next = None  # Pointer to the next guest in the linked list

    def __repr__(self):
        return f"Guest({self.name})"

    def set_next(self, next_guest):
        self.next = next_guest

    def get_next(self):
        return self.next

    def has_next(self):
        return self.next is not None

    def get_name(self):
        return self.name

    def __str__(self):
        return self.name()


class HotelAlphabetical_SOLUTION:
    """Class representing a hotel where guests are stored in rooms based on
    the first initial of their names. Each room is a linked list of guests.
    """

    _DEFAULT_CAPACITY = 26
    _ASCII_LEFT_EDGE = ord("A")
    _ASCII_RIGHT_EDGE = ord("Z")

    _LOAD_FACTOR_THRESHOLD = 0.7
    _INCREMENT_FACTOR = 2

    def __init__(self, capacity: int = _DEFAULT_CAPACITY):
        self._capacity = capacity
        self._hotel = [None] * capacity  # Array of linked lists for each letter
        self._usage = 0  # number of array slots used
        self._size = 0

    def _get_index(self, name: str) -> int:
        """Compute the index in the hotel array based on the first
        initial of the guest's name."""
        # Default to 0 if name is None or empty or not A-Z
        room_index = 0
        if name is not None and len(name) > 0:
            # DISCUSSION POINT: should we be computing the first initial
            # here or should it be done in object Guest?
            initial_ascii = ord(name.upper()[0])
            if self._ASCII_LEFT_EDGE <= initial_ascii <= self._ASCII_RIGHT_EDGE:
                room_index = initial_ascii % self._capacity
        return room_index

    def _check_load_factor(self) -> bool:
        """Check if the load factor exceeds the threshold."""
        load_factor = self._usage / self._capacity
        return load_factor > self._LOAD_FACTOR_THRESHOLD

    def _rehash(self) -> None:
        """Rehash the hotel by increasing its capacity and reassigning guests."""
        # Preserve the old hotel array and its capacity
        old_hotel = self._hotel
        old_capacity = self._capacity
        # Create a new hotel array with increased capacity
        self._capacity *= self._INCREMENT_FACTOR
        # Initialize the new hotel array and reset usage
        self._hotel = [None] * self._capacity
        self._usage = 0
        # Reinsert all guests into the new hotel array
        for room in range(old_capacity):
            guest_in_room = old_hotel[room]
            while guest_in_room is not None:
                self.add_guest(guest_in_room.get_name())
                guest_in_room = guest_in_room.get_next()

    def add_guest(self, name: str) -> None:
        """Add a guest to the hotel."""
        if self._check_load_factor():
            self._rehash()
        # Compute the room index based on the first initial of the name
        room = self._get_index(name)
        # Create a new guest object
        guest = Guest(name)
        # Insert the guest at the front of the linked list for that room
        if self._hotel[room] is None:
            self._hotel[room] = guest
            self._usage += 1
        else:
            guest.set_next(self._hotel[room])
            self._hotel[room] = guest
        # Increment the current occupancy of the hotel
        self._size += 1

    def exists(self, name) -> bool:
        """Check if a guest with the given name exists in the hotel."""
        # Initialize a return variable expecting to not find the name
        # we are looking for
        found = False
        # Determine the expected room based on the first initial of the name
        expected_room = self._get_index(name)
        # Traverse the linked list in that room to find the guest
        guest_in_room = self._hotel[expected_room]
        while guest_in_room is not None and not found:
            found = guest_in_room.get_name() == name
            guest_in_room = guest_in_room.get_next()
        return found

    def remove(self, name) -> Guest | None:
        """Remove a guest with the given name from the hotel."""
        # Initialize a return variable expecting to not find the name
        removed_guest = None
        # Determine the expected room based on the first initial of the name
        expected_room = self._get_index(name)
        # Traverse the linked list in that room to find and remove the guest.
        # When we find the guest to be removed, we need to update the pointers
        # accordingly, so that the next guest of the previous guest points to
        # the next guest of the current guest (the one being removed).  If
        # the guest to be removed is the first guest in the room, we need to
        # update the head of the linked list for that room.
        current_guest = self._hotel[expected_room]
        previous_guest = None
        while current_guest is not None and removed_guest is None:
            if current_guest.get_name() == name:
                removed_guest = current_guest
                if previous_guest is None:
                    # Removing the first guest in the room
                    self._hotel[expected_room] = current_guest.get_next()
                else:
                    # Bypass the current guest
                    previous_guest.set_next(current_guest.get_next())
                # No matter what, we have removed a guest from the hotel
                # and need to decrement the size.
                self._size -= 1
            else:
                # Move to the next guest in the linked list
                # remembering the previous one.
                previous_guest = current_guest
                current_guest = current_guest.get_next()
        return removed_guest

    # Constants for string representation
    _EMPTY = "boohoo, your hotel is empty."
    _NEXT_GUEST = " --> "
    _FMT_GUESTS = f"There are {{}} guest(s) in your hotel."
    _FMT_CAPACITY = f"The hotel has a capacity of {{}} rooms."
    _FMT_USAGE = f"The hotel is using {{}} room(s)."
    _FMT_LOAD_FACTOR = f"The load factor is {{:.2f}}."
    _FMT_GUEST_LIST = f"The {{}} guest(s) are:"
    _FMT_ROOM = f"Room {{:02d}}: "

    def __repr__(self) -> str:
        hotel_string = self._EMPTY
        if self._size > 0:
            hotel_string = f"\n{self._FMT_GUESTS.format(self._size)}"
            hotel_string += f"\n{self._FMT_CAPACITY.format(self._capacity)}"
            hotel_string += f"\n{self._FMT_USAGE.format(self._usage)}"
            hotel_string += (
                f"\n{self._FMT_LOAD_FACTOR.format(self._usage/self._capacity)}"
            )
            hotel_string += f"\n{self._FMT_GUEST_LIST.format(self._size)}"
            for room in range(self._capacity):
                if self._hotel[room] is not None:
                    hotel_string += f"\n\t{self._FMT_ROOM.format(room)}"
                    guest_in_room = self._hotel[room]
                    while guest_in_room is not None:
                        hotel_string += f"{guest_in_room.get_name()}{self._NEXT_GUEST}"
                        guest_in_room = guest_in_room.get_next()
                    hotel_string += ""
        return hotel_string